# The Battle of Neighborhoods
   
   ###  Course: Applied Data Science Capstone
   
This notebook would contain Explaination and code for the coursera Capstone Project. 


   ### Adding Required Libraries to create required Environment

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.expand_frame_repr', False)

First Print Statement !! :)


In [ ]:
print('Hello Capstone Project Course!')


# Segmenting and Clustering Neighborhoods in Toronto

## 1. Data scraping

#### Description: 
Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.


In [ ]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
nbr = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [ ]:
nbr

#### Web scraping
##### Importing Data From provided URL

In [ ]:
#import the library used to query a website
from urllib.request import urlopen

#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Query the website and return the wiki to the variable 'page'
page = urlopen(wiki)

#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page, "lxml")


In [ ]:
# Uncomment below to Print HTML of the Page
#print(soup.prettify()) 

#Get table 
right_table=soup.find('table', class_='wikitable sortable')
# uncomment below to print table
#right_table  

##### Extract the information to DataFrame: 
we iterate through each row (tr) and then assign each element of tr (td) to a variable and append it to a list.

In [ ]:
#Generate lists
A=[]
B=[]
C=[]

for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

        
#Adding Data to our DataFrame
nbr['Postal_Code']=A
nbr['Borough']=B
nbr['Neighborhood']=C

#uncomment below to view dataframe nbr which contains data as retreived from the webpage in wiki
#nbr

####  Ignore cells with a borough that is Not assigned.
1. Compare with Each row if Borough is Not Assigned drop row.
2. Reset Index.

In [ ]:
nbr = nbr.drop(nbr[nbr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

#Reset Index
nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe where not Assigned Boroughs have been dropped out.
#nbr               

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
1. instantiate additional dataframe nbrNA
2. Compare with Each row if Neighborhood is Not Assigned add value from Borough.

In [ ]:
# instantiate additional dataframe nbrNA
column_n = ['NBRVAL'] 
nbrNA = pd.DataFrame(columns=column_n)
D=['Not assigned'] 
nbrNA['NBRVAL'] = D

#Dataframe contains 'Not assigned' value used for Comparison.
#print(nbrNA)

nbr1=nbr
#nbr1


#Compare with Each row if Neighborhood is Not Assigned add value from Borough.
for row_index,row in nbr.iterrows():
    if((nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
        #print('inside if')
        nbr1.loc[row_index,['Neighborhood']] = nbr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe nbr1 which contains Neighborhoods which were initially 'Not Assigned' now containing Borough values
print('nbr1')
nbr1               

##### More than one neighborhood can exist in one postal code area.

In [ ]:
nbr2=nbr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()
#Uncomment Below to view nbr2 which contains grouped Boroughs
nbr2

#### Printing the number of rows of neighborhoods dataframe.

In [ ]:
print(nbr2.shape)
print('nbr2')
nbr2

## Filtering Neighborhood
Creating a copy of Dataframe.
Dropping neighborhoods which do not Belong to Toronto

In [ ]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
neighbor = pd.DataFrame(columns=column_names)

neighbor = nbr2.drop(nbr2[nbr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
neighbor.index = pd.RangeIndex(len(neighbor.index))
print('neighbor')
print(neighbor)

In [ ]:
#Keeping nbr_ungrp for ease of location calculations, using nbr1 by dropping Non Toronto Boroughs.
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
nbr_ungrp = pd.DataFrame(columns=column_names)

nbr_ungrp = nbr1.drop(nbr1[nbr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))

#Uncomment below to view Dataframe
print(nbr_ungrp)

### Capturing Longitude And Latitude info of Locations
using the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html

In [ ]:
# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes 
# convert an address into latitude and longitude values
import time
from geopy.geocoders import Nominatim 

#### Looping through locations to fetch Latitude and Longitude.

In [ ]:
geolocator = Nominatim(user_agent="my_app app123")
#print('here')

for row_index, item in nbr_ungrp.iterrows():
    
    list1 = nbr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    #print(loc)
    #print(row_index)
    time.sleep(5)
    if(location is not None):
        #print(location)
        #print("location Address : ")
        #print(location.address)
        #print('latitude, longitude : ')
        #print((location.latitude, location.longitude))
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Latitude'] = location.latitude
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Longitude'] = location.longitude
        #print(nbr_ungrp[row_index])    

print(nbr_ungrp)


# Neighborhood analysis#
# Importing Libraries for Foursqaue to analyse Neighborhood.
    


In [ ]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Foursquare and map plotting Libraries imported.')


#### Number of Boroughs and Neighborhoods

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nbr_ungrp['Borough'].unique()),
        nbr_ungrp.shape[0]
    )
)

#Dropping NANs
nbr_ungrp.dropna(inplace =True)
#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))
nbr_ungrp


#### Using geopy library to get the latitude and longitude values of Toronto City.

In [ ]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="my-app123")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

## Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nbr_ungrp['Latitude'], nbr_ungrp['Longitude'], nbr_ungrp['Borough'], nbr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#### Define Foursquare Credentials and Version

### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's details


In [ ]:
nbr_ungrp.loc[0]
neighborhood_latitude = np.float(nbr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(nbr_ungrp.loc[0,['Longitude']].values)

#### Now, let's get the top 25 venues that are in Harbourfront within a radius of 500 meters.
First, let's create the GET request URL. Naming the URL url.

In [ ]:
# The code was removed by Watson Studio for sharing.

####  get_category_type function

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

cleaning the json and structuring it into a pandas dataframe.

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

 how many venues were returned by Foursquare?

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

### Exploring all Neighborhoods-


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
### Getting Toronto Venues


In [ ]:
toronto_venues = getNearbyVenues(names=nbr_ungrp['Neighborhood'],
                                   latitudes=nbr_ungrp['Latitude'],
                                   longitudes=nbr_ungrp['Longitude']
                                  )




Size of dataframe

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

Let's check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

## 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 7 venues for each neighborhood.

In [ ]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 4 clusters.

In [ ]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
toronto_merged = nbr_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
#### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]